# European Option Pricing

Within this page, two methods are used to price options, one being the Black-Scholes Formula, and the other being a Monte Carlo Simulation method. To complete this, the following steps are taken:
1. Gather historical and some current option data
2. Run Monte Carlo for pricing
3. Price using Monte Carlo Results and Black-Scholes Formula

In [361]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from scipy.stats import norm

# Gathering Data

First we must get the risk-free rate. I used the US Treasury Dept. The rate can be found at the link in the next cell. I get the rate for one month as I am pricing for options with an expiration date in about a month. We then covert this to a compoud risk-free rate.

After this, we get the data through yfinance. This package provides enough for options strike price and implied volatility, which is pretty mush all we need. We then use the strike price closest to the current closing price for options that expire on the date closest to one month away. From this, we also get the implied volatility. 

https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2025

In [362]:
month_1_rate = 	4.06/100
compound_rate = np.log(1 + month_1_rate)
total = 1 * 365

In [363]:
ticker = input("What stock would you like to predict ->").upper()
stock = yf.Ticker(ticker)
date_start = dt.date.today() - dt.timedelta(days = total)
data = stock.history(start = date_start)
data = data.drop(columns=['Volume','Dividends','Stock Splits'])

data["Percent Change"] = data["Close"].pct_change() 
data["Log Change"] = np.log(data['Percent Change'] + 1) #normal dist

target = dt.date.today() + dt.timedelta(days= 30)
closest_date = stock.options[np.argmin(np.abs(np.array(stock.options,dtype='datetime64[D]').astype(dt.datetime) - target))]
options_data = stock.option_chain(closest_date) #options from the earliest expiration

calls = options_data.calls.copy()
puts = options_data.puts.copy()

In [ ]:
last_close = data['Close'][-1]


closest_call = calls['strike'].iloc[np.argmin(np.abs(calls['strike'] - last_close))]
closest_call_implied = calls['impliedVolatility'].iloc[np.argmin(np.abs(calls['strike'] - last_close))]
closest_call_price = calls['lastPrice'].iloc[np.argmin(np.abs(calls['strike'] - last_close))]


closest_put = puts['strike'].iloc[np.argmin(np.abs(puts['strike'] - last_close))]
closest_put_implied = puts['impliedVolatility'].iloc[np.argmin(np.abs(puts['strike'] - last_close))]
closest_put_price = puts['lastPrice'].iloc[np.argmin(np.abs(puts['strike'] - last_close))]


trade_years = (np.asarray(closest_date,dtype='datetime64[D]').astype(dt.datetime) - data.index[-1].date()) / dt.timedelta(365)

2025-12-05 305.0 0.3616091671752929 13.84
2025-12-05 305.0 0.21442435974121093 5.7


# Functions to For Monte Carlo and Black-Scholes

## Monte Carlo

This Monte Carlo is very similar to one of Geometric Brownian Motion, with some slight changes. This time the formula for end prices are $$ S_T = S_0 * e^{drift * T + \sigma  \sqrt{T}Z} $$
Here, $ drift = r - \frac{1}{2} \sigma^2 $ where $ r = \text{Compund Risk Rree Rate} $ and $ \sigma = \text{Implied Volatility of Stock}$

T is the time in years, which will be $ T = \frac{\text{Days until expiration}}{\text{Days in a year}} $

Lastly, $Z$ is sample from a standard normal distribution.

## Pricing
Let $K = \text{The Strike Price}$ and $R = \text{The results of the Monte Carlo Simulation}$.

The the method to price Call options is as follows. First we cacluate the expected payoff as $ Y = E[\max(R - K, 0)] $. Lastly, the price at returned to a discounted to present value using $e^{-r * Y}$.

For Put options, the formula is the same, except the expected payoffs, $ Y = E[\max(K - R, 0)] $ Then this is taken to discounted price using the same formula as Call Options. 

## Black-Scholes
Let $K = \text{The Strike Price}$

$C = \text{The Last Close}$

$T = \text{Time, as defined as above}$

$r = \text{Compound Risk Free Rate}$

$\sigma = \text{The Implied Volatility of the Stock}$

$N(x) = \text{A Standard Normal Distribution} $

Then the Black-Scholes Formula for a Call Option: $$\text{Price =} C N(d_1) - K e^{-rT}N(d_2)$$

and for Put Option: $$\text{Price = } N(-d_2) K e^{-rT} - N(-d_1) C $$

Where $$d_1 = \frac{\log{\frac{C}{K}} + r + \frac{1}{2} \sigma^2T}{\sigma \sqrt{T}} $$ and $$d_2 = d_1 - \sigma \sqrt{T} $$

The Black-Scholes Model works with a couple of assumptions:
1. Prices are Log-Normal Distributed
2. Returns are Normal Distributed
3. Volitality is constant.

In [365]:
def GBM_risk_neutral(data,implied_volatility, num = 10000, r = compound_rate, years = 1.0):
    std = implied_volatility #annual
    drift = r - (1/2) * std**2 #annual

    results =  data['Close'][-1] * np.exp(np.random.normal(loc= drift * years, scale= std * np.sqrt(years), size = num))
    return results

def pricing(results, K, r = compound_rate, years = 1.0, mode = 'call'):
    if mode == 'call':
        all_end = np.maximum(results - K, 0)
        call_payoff = np.mean(all_end)
        return np.exp(-r * years) * call_payoff
    elif mode == 'put':
        all_end = np.maximum(K - results, 0)
        put_payoff = np.mean(all_end)
        return np.exp(-r * years) * put_payoff
    else:
        print("Mode must be 'call' or 'put'")

def black_scholes(last_close, sigma, K, r = compound_rate, years = 1.0, mode = 'call'):
    if mode == 'call':
        d1 = (np.log(last_close/K) + (r + (1/2) * sigma **2) * years) / (sigma * np.sqrt(years)) 
        d2 = d1 - sigma * np.sqrt(years)
        call_price = last_close * norm.cdf(d1) - norm.cdf(d2) * K * np.exp(-r * years)
        return call_price
    else:
        d1 = (np.log(last_close/K) + (r + (1/2) * sigma **2) * years) / (sigma * np.sqrt(years))
        d2 = d1 - sigma * np.sqrt(years)
        put_price = norm.cdf(-d2) * K * np.exp(-r * years) - norm.cdf(-d1) * last_close
    return put_price

# Pricing and Repots

Below, the pricing models are run, and the findings are reported following the models.

In [366]:
sim_call_implied = GBM_risk_neutral(data, years = trade_years, implied_volatility = closest_call_implied)
sim_put_implied = GBM_risk_neutral(data, years= trade_years, implied_volatility = closest_put_implied)

MC_call_price = pricing(sim_call_implied, closest_call, compound_rate, trade_years, mode= 'call')
MC_put_price = pricing(sim_put_implied, closest_put, compound_rate, trade_years, mode= 'put')
BS_call_price = black_scholes(last_close,closest_call_implied, closest_call,compound_rate,trade_years,'call')
BS_put_price = black_scholes(last_close,closest_put_implied, closest_put,compound_rate,trade_years,'put')

In [367]:
print(f"Call prices for {ticker} on {closest_date} at rate {month_1_rate * 100}, and strike {closest_call}:")
print(f"Monte Carlo: {round(MC_call_price,2)}")
print(f'Black-Scholes: {round(BS_call_price,2)} ')

print()
print(f"Put prices for {ticker} on {closest_date} at rate {month_1_rate * 100} and strike {closest_put}:")
print(f"Monte Carlo: {round(MC_put_price,2)}")
print(f'Black-Scholes: {round(BS_put_price,2)}')

print(f"\nActual Prices on Yahoo Finance: Call - {closest_call_price}, Put - {closest_put_price}")

Call prices for JPM on 2025-12-05 at rate 4.06, and strike 305.0:
Monte Carlo: 14.85
Black-Scholes: 14.89 

Put prices for JPM on 2025-12-05 at rate 4.06 and strike 305.0:
Monte Carlo: 6.09
Black-Scholes: 6.1

Actual Prices on Yahoo Finance: Call - 13.84, Put - 5.7
